### 12.09 testing 2.01

In [1]:
# import part
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Dropout,  UpSampling2D
from keras.optimizers import RMSprop
from keras import backend as K
import matplotlib.pyplot as plt

# loading data
X_train = np.loadtxt("1209.2_spec_train_1000.txt")
y_train = np.loadtxt("1209.2_mask_train_1000.txt")

X_test = np.loadtxt("1209.2_spec_test_1000.txt")
y_test = np.loadtxt("1209.2_mask_test_1000.txt")


/home/william/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# No hiden layer. just fully connected
# sigmoid as activation function
X_train = X_train.reshape(-1,1,2,512)
y_train = y_train.reshape(-1,1,2,512)

model = Sequential([
    #Dense(1024, input_dim=1024),
    #Activation('relu'), 
    Conv2D(16,(3,3),activation='relu',padding='same')()
    UpSampling2D(1024, size=(2, 2), data_format=None, interpolation='nearest'),
    
    
    Activation('hard_sigmoid'),  # ,sigmoid softmax
    #Dense(247),
    #Activation('relu'),
    #Dense(2470),
    #Activation('sigmoid')
])

TypeError: __init__() got multiple values for argument 'size'

In [ ]:
# optimizer

rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-8, decay=0.0)


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


model.compile(
    optimizer=rmsprop,
    loss='mean_squared_error',
    metrics=['accuracy']  # , mean_pred]
)
print('Train-------------------------')

history = model.fit(X_train, y_train, validation_split=0.25, epochs=100, shuffle = True, batch_size=75, verbose=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output1 = get_3rd_layer_output([X_train])[0]

print(layer_output1)

In [ ]:
y_train.size



In [ ]:
y_mask = y_train.reshape(1000,1024)
plt.imshow(abs(y_mask[:, : int(512 / 2 + 1)].T), aspect = "auto", cmap=plt.cm.afmhot, origin = "lower")
plt.title("Lable_Mask", fontsize = 20)
plt.show()

In [ ]:
x_pred = model.predict(X_train[0:1000])
print('prediction of the model', x_pred)
print('prediction size', x_pred.size)

x_mask = x_pred.reshape(1000,1024)
plt.imshow(abs(x_mask[:, : int(512 / 2 + 1)].T), aspect = "auto", cmap=plt.cm.afmhot, origin = "lower")
plt.title("Predicted_Training_Mask", fontsize = 20)
plt.show()

In [ ]:
y_mask = y_test.reshape(1000,1024)
plt.imshow(abs(y_mask[:, : int(512 / 2 + 1)].T), aspect = "auto", cmap=plt.cm.afmhot, origin = "lower")
plt.title("Test_Lable_Mask", fontsize = 20)
plt.show()

In [ ]:
x_pred = model.predict(X_test[0:1000])
print('prediction of the model', x_pred)
print('prediction size', x_pred.size)

x_mask = x_pred.reshape(1000,1024)
plt.imshow(abs(x_mask[:, : int(512 / 2 + 1)].T), aspect = "auto", cmap=plt.cm.afmhot, origin = "lower")
plt.title("Test_Predicted_Mask", fontsize = 20)
plt.show()